In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Activation
#from tensorflow.keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import time
import numpy as np
import cv2
from numpy import newaxis
import time
from sklearn.utils import resample
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix
import gc


In [2]:
def predimg(model, imgs, bs):
    #print(imrs)
    #start_time=time.time()

    preds = model.predict(imgs)

    #print(preds)
    predicted_class = np.argmax(preds, axis=1)
    #print(predicted_class)
    prob = np.empty(bs)
    score1 = np.empty(bs)
    for i in range(bs):
        prob[i] = preds[i][predicted_class[i]]
        score1[i] = preds[i][1]

    #print(prob)
    return predicted_class, prob, score1

In [3]:
import pickle
class_indices = pickle.load( open( "testnnv2-onenewnetv2.p", "rb" ) )
print(class_indices)

class_inv_map = {v: int(k) for k, v in class_indices.items()}

print(class_inv_map)

{'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12}
{0: 0, 1: 1, 2: 10, 3: 11, 4: 12, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6, 10: 7, 11: 8, 12: 9}


In [4]:
import glob

    
from tensorflow.keras.models import load_model
model = load_model('one/0_ov4nnnewnet_f0.8_d0.5_l0.01.h5')
#model = load_model('nnparasnet_f32_d0.2_l0.001.h5')


In [5]:
def genresult(X_test):
    bsize = 32
    #print(X_test)
    flen = len(X_test)

    rlen = flen//bsize
    #rlen = 1

    rst = flen - rlen * bsize

    print('Lens:', flen, 'Blocks:', rlen, 'Bsize:', bsize, 'Rst:', rst)

    t1 = time.time()
    data = np.zeros((bsize, 120,120,3), dtype=np.float32)

    if rst:
        rlen = rlen + 1
    y_pred = np.array([])
    y_prob = np.array([])
    y_score1= np.array([])
    for i in range(rlen):
        bs = bsize
        if rst and i == (rlen -1):
            bs = rst
        for j in range(bs):
            fid = i*bsize + j
            image = cv2.imread(X_test[fid])
            imrs = image.astype(float)
            imrs = imrs / 255.0
            data[j,:,:,:] = imrs

        #print('i:', i, 'bs', bs)
        ret, prob, score1 = predimg(model, data.copy(), bs)

        y_pred = np.hstack((y_pred, ret[:bs]))
        y_prob = np.hstack((y_prob, prob[:bs]))
        y_score1 = np.hstack((y_score1, score1[:bs]))
        #print(ret)
        #print(prob)
        #zzz
    t2 = time.time()

    ttime = t2 - t1
    #print('All:', len(y_pred))
    return y_pred, y_prob

In [6]:
opath = './testdemo/'

In [7]:
labels = ['Contaminations', 'Cryptosporidium', 'Giardia', '1.54um beads', '3um beads', '4um beads', '4.6um beads', '5um beads', '5.64um beads', '8um beads', '10um beads', '12um beads', '15um beads']


In [8]:
print(class_inv_map)

{0: 0, 1: 1, 2: 10, 3: 11, 4: 12, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6, 10: 7, 11: 8, 12: 9}


In [13]:
oret={}

data = glob.glob(opath + '/*.png')
dlen = len(data)

X_test =[]
X_test = X_test + data

y_pred, y_prob = genresult(X_test)
print('++++')
y_predt = np.vectorize(class_inv_map.get)(y_pred)

for i in range(dlen):
    print('File:', data[i] , '\t\tPredict: ', labels[y_predt[i]], '\tConfidence: ', y_prob[i])
    print()
    print()



Lens: 21 Blocks: 0 Bsize: 32 Rst: 21
++++
File: ./testdemo/c_4.png 		Predict:  Cryptosporidium 	Confidence:  0.995836615562439


File: ./testdemo/ct_12.png 		Predict:  Contaminations 	Confidence:  0.9999997615814209


File: ./testdemo/g_2.png 		Predict:  Giardia 	Confidence:  0.9999046325683594


File: ./testdemo/c_3.png 		Predict:  Cryptosporidium 	Confidence:  0.9998962879180908


File: ./testdemo/ct_9.png 		Predict:  Contaminations 	Confidence:  1.0


File: ./testdemo/ct_8.png 		Predict:  Contaminations 	Confidence:  0.9999792575836182


File: ./testdemo/15_1.png 		Predict:  15um beads 	Confidence:  1.0


File: ./testdemo/ct_2.png 		Predict:  Contaminations 	Confidence:  0.9999996423721313


File: ./testdemo/c_1.png 		Predict:  Cryptosporidium 	Confidence:  0.9999768733978271


File: ./testdemo/ct_11.png 		Predict:  Contaminations 	Confidence:  0.9999213218688965


File: ./testdemo/ct_3.png 		Predict:  Contaminations 	Confidence:  0.9999994039535522


File: ./testdemo/ct_4.png 		Pre

In [ ]:

del model
gc.collect()